In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# load train and test data (already splited)
train_df = pd.read_csv('./data/train_df.csv')
test_df = pd.read_csv('./data/test_df.csv')

# data preprocessing function
def normalize_data(arr):
    S = sum(arr)
    return [x / S for x in arr]

# train dataset preprocessing
X_train = train_df.drop(columns=['km_diagnosis'])
y_train = train_df['km_diagnosis']

# normalize columns that are not in 0~1 range
columns_to_normalize = ['bmi', 'forehead_circumference', 'neck_circumference', 
                        'armpit_circumference', 'bust', 'rib_cage', 
                        'waist_circumference', 'iliac_circumference', 
                        'femur_circumference', 'urinenighttime_urination']

for col in columns_to_normalize:
    X_train[col] = normalize_data(X_train[col].values)

# test dataset preprocessing
X_test = test_df.drop(columns=['km_diagnosis'])
y_test = test_df['km_diagnosis']

for col in columns_to_normalize:
    X_test[col] = normalize_data(X_test[col].values)


# 로지스틱 회귀 모델 학습
# 초기 가중치 설정 (동형암호 모델과 일치하도록)
initial_beta = 2 * np.random.rand(X_train.shape[1]) - 1
model = LogisticRegression(C=1e5, solver='lbfgs', max_iter=1000, fit_intercept=True)
model.coef_ = np.array([initial_beta[:-1]])
model.intercept_ = np.array([initial_beta[-1]])

model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_test)

# 정확도 계산
accuracy = accuracy_score(y_test, y_pred)
print(f"평문 로지스틱 회귀 모델의 정확도: {accuracy}")

# 예측 확률을 사용하여 임계값(0.6)을 적용한 후, 조정된 정확도 계산
y_pred_proba = model.predict_proba(X_test)[:, 1]
threshold = 0.6
y_pred_adjusted = (y_pred_proba >= threshold).astype(int)
adjusted_accuracy = accuracy_score(y_test, y_pred_adjusted)
print(f"임계값을 조정한 평문 로지스틱 회귀 모델의 정확도: {adjusted_accuracy}")

평문 로지스틱 회귀 모델의 정확도: 0.5466666666666666
임계값을 조정한 평문 로지스틱 회귀 모델의 정확도: 0.5466666666666666
